# Dataframe manipulation with tidyr


Researchers often want to manipulate their data from the ‘wide’ to the ‘long’ format, or vice-versa.  

**Identifier variables**: Identifier, or ID, variables identify the observations. Think of these as the **key(s)** that identifies your observations. 

**Observed/Measured variables**: This represents the measurements you observed.

In the **long** format, there is usually 1 column for the observed variables and other column(s) are ID variables. 

In the **wide** format, there is usually a column for each observed variable. 

Many (but not all!) of R’s functions have been designed assuming you have ‘long’ format data. Howerver for humans, format affects readability. Wide format is often more intuitive since we can see more of the data on the screen. Long format is more machine readable and is closer to the formatting of databases. 
**This tutorial will help you efficiently transform your data regardless of original format.**

![](https://swcarpentry.github.io/r-novice-gapminder/fig/14-tidyr-fig1.png)

## Getting started

First install the packages if you haven’t already done so (you probably installed dplyr in the previous lesson); then load the packages.

In [1]:
#install.packages("tidyr")
#install.packages("dplyr")
library("tidyr")
library("dplyr")


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



First, lets look at the structure of our original gapminder dataframe using str()

In [2]:
gapminder <- read.csv("../data/gapminder_data.csv", stringsAsFactor = FALSE)
str(gapminder)
head(gapminder)

'data.frame':	1704 obs. of  6 variables:
 $ country  : chr  "Afghanistan" "Afghanistan" "Afghanistan" "Afghanistan" ...
 $ continent: chr  "Asia" "Asia" "Asia" "Asia" ...
 $ year     : int  1952 1957 1962 1967 1972 1977 1982 1987 1992 1997 ...
 $ lifeExp  : num  28.8 30.3 32 34 36.1 ...
 $ pop      : int  8425333 9240934 10267083 11537966 13079460 14880372 12881816 13867957 16317921 22227415 ...
 $ gdpPercap: num  779 821 853 836 740 ...


country,continent,year,lifeExp,pop,gdpPercap
Afghanistan,Asia,1952,28.801,8425333,779.4453
Afghanistan,Asia,1957,30.332,9240934,820.8530
Afghanistan,Asia,1962,31.997,10267083,853.1007
Afghanistan,Asia,1967,34.020,11537966,836.1971
Afghanistan,Asia,1972,36.088,13079460,739.9811
Afghanistan,Asia,1977,38.438,14880372,786.1134


### Question 1
**Is gapminder a purely long, purely wide, or some intermediate format?**  
Ans. The original gapminder data.frame is in an intermediate format. It is not purely long since it had multiple observation variables (ex. pop, lifeExp, gdpPercap).



Sometimes, as with the gapminder dataset, we have multiple types of observed data. It is somewhere in between the purely long and wide data formats. We have 3 ID variables (continent, country, year) and 3 “Observation variables” (pop,lifeExp,gdpPercap). This is often preferred due to readability, flexibility when analyzing, and units being on different scales. 

While using many of the functions in R you usually do not want to do mathematical operations on values with different units. For example, using the purely long format, a single mean for all of the values of population, life expectancy, and GDP would not be meaningful. The solution would be to restructure your dataframe or use a dplyr function for grouping. 

## From wide to long format with tidyr::gather()

Until now, we’ve been using the nicely formatted original gapminder dataset, but ‘real’ data (i.e. our own research data) will never be so well organized. Here let’s start with the wide format version of the gapminder dataset. 

**Download/read in the wide version of the gapminder data.**


We’ll load the data file and look at it. Note: we don’t want our continent and country columns to be factors, so we use the stringsAsFactors argument for read.csv() to disable that.

In [3]:
gap_wide <- read.csv("../data/gapminder_wide.csv", stringsAsFactors = FALSE)

head(gap_wide)

country,continent,gdpPercap_1952,gdpPercap_1957,gdpPercap_1962,gdpPercap_1967,gdpPercap_1972,gdpPercap_1977,gdpPercap_1982,gdpPercap_1987,⋯,pop_1962,pop_1967,pop_1972,pop_1977,pop_1982,pop_1987,pop_1992,pop_1997,pop_2002,pop_2007
Afghanistan,Asia,779.4453,820.853,853.1007,836.1971,739.9811,786.1134,978.0114,852.3959,⋯,10267083,11537966,13079460,14880372,12881816,13867957,16317921,22227415,25268405,31889923
Albania,Europe,1601.0561,1942.284,2312.8890,2760.1969,3313.4222,3533.0039,3630.8807,3738.9327,⋯,1728137,1984060,2263554,2509048,2780097,3075321,3326498,3428038,3508512,3600523
Algeria,Africa,2449.0082,3013.976,2550.8169,3246.9918,4182.6638,4910.4168,5745.1602,5681.3585,⋯,11000948,12760499,14760787,17152804,20033753,23254956,26298373,29072015,31287142,33333216
Angola,Africa,3520.6103,3827.940,4269.2767,5522.7764,5473.2880,3008.6474,2756.9537,2430.2083,⋯,4826015,5247469,5894858,6162675,7016384,7874230,8735988,9875024,10866106,12420476
Argentina,Americas,5911.3151,6856.856,7133.1660,8052.9530,9443.0385,10079.0267,8997.8974,9139.6714,⋯,21283783,22934225,24779799,26983828,29341374,31620918,33958947,36203463,38331121,40301927
Australia,Oceania,10039.5956,10949.650,12217.2269,14526.1246,16788.6295,18334.1975,19477.0093,21888.8890,⋯,10794968,11872264,13177000,14074100,15184200,16257249,17481977,18565243,19546792,20434176


![](https://swcarpentry.github.io/r-novice-gapminder/fig/14-tidyr-fig2.png)


The first step towards getting our nice intermediate data format is to convert from the wide to the long format. The tidyr function gather() will ‘gather’ your observation variables into a single variable.



![](https://swcarpentry.github.io/r-novice-gapminder/fig/14-tidyr-fig3.png)

In [4]:
gap_long <- gap_wide %>%
    gather(key = "obstype_year", value = "obs_values", starts_with('pop'),
           starts_with('lifeExp'), starts_with('gdpPercap'))
head(gap_long)

country,continent,obstype_year,obs_values
Afghanistan,Asia,pop_1952,8425333
Albania,Europe,pop_1952,1282697
Algeria,Africa,pop_1952,9279525
Angola,Africa,pop_1952,4232095
Argentina,Americas,pop_1952,17876956
Australia,Oceania,pop_1952,8691212


Here we have used piping syntax which is similar to what we were doing in the previous lesson with dplyr. In fact, these are compatible and you can use a mix of tidyr and dplyr functions by piping them together.

Inside gather() we first name the new column for the new ID variable (obstype_year), the name for the new amalgamated observation variable (obs_value), then the names of the old observation variable. We could have typed out all the observation variables, but as in the select() function (see dplyr lesson), we can use the starts_with() argument to select all variables that starts with the desired character string. Gather also allows the alternative syntax of using the - symbol to identify which variables are not to be gathered (i.e. ID variables)

![](https://swcarpentry.github.io/r-novice-gapminder/fig/14-tidyr-fig4.png)

In [5]:
gap_long <- gap_wide %>% gather(key = "obstype_year", value = "obs_values",-continent,-country)

head(gap_long)
str(gap_long)

country,continent,obstype_year,obs_values
Afghanistan,Asia,gdpPercap_1952,779.4453
Albania,Europe,gdpPercap_1952,1601.0561
Algeria,Africa,gdpPercap_1952,2449.0082
Angola,Africa,gdpPercap_1952,3520.6103
Argentina,Americas,gdpPercap_1952,5911.3151
Australia,Oceania,gdpPercap_1952,10039.5956


'data.frame':	5112 obs. of  4 variables:
 $ country     : chr  "Afghanistan" "Albania" "Algeria" "Angola" ...
 $ continent   : chr  "Asia" "Europe" "Africa" "Africa" ...
 $ obstype_year: chr  "gdpPercap_1952" "gdpPercap_1952" "gdpPercap_1952" "gdpPercap_1952" ...
 $ obs_values  : num  779 1601 2449 3521 5911 ...


That may seem trivial with this particular dataframe, but sometimes you have 1 ID variable and 40 Observation variables with irregular variables names. The flexibility is a huge time saver!

Now obstype_year actually contains 2 pieces of information, the observation type (pop,lifeExp, or gdpPercap) and the year. We can use the **separate()** function to split the character strings into multiple variables.

In [6]:
gap_long <- gap_long %>% separate(obstype_year,into=c('obs_type','year'),sep="_")
gap_long$year <- as.integer(gap_long$year)

### Question 2
**Using gap_long, create a line graph using ggplot showing the gdpPercap trend over time for Canada. Hint: use the filter and ggplot functions.**

In [29]:
library(ggplot2)
#gap_long %>% filter(country == "Canada" & obs_type == "gdpPercap") %>% ggplot2::ggplot(aes(x = year, y = obs_values)) + geom_line()

## From long to intermediate format with tidyr::spread()


Now let’s use the opposite of gather(), to spread our observation variables back out with the aptly named spread(). We can then spread our gap_long() to the original intermediate format or the widest format. Let’s start with the intermediate format.

In [7]:
gap_normal <- gap_long %>% spread(key = "obs_type", value = "obs_values")

head(gap_normal)
head(gapminder)

country,continent,year,gdpPercap,lifeExp,pop
Afghanistan,Asia,1952,779.4453,28.801,8425333
Afghanistan,Asia,1957,820.8530,30.332,9240934
Afghanistan,Asia,1962,853.1007,31.997,10267083
Afghanistan,Asia,1967,836.1971,34.020,11537966
Afghanistan,Asia,1972,739.9811,36.088,13079460
Afghanistan,Asia,1977,786.1134,38.438,14880372


country,continent,year,lifeExp,pop,gdpPercap
Afghanistan,Asia,1952,28.801,8425333,779.4453
Afghanistan,Asia,1957,30.332,9240934,820.8530
Afghanistan,Asia,1962,31.997,10267083,853.1007
Afghanistan,Asia,1967,34.020,11537966,836.1971
Afghanistan,Asia,1972,36.088,13079460,739.9811
Afghanistan,Asia,1977,38.438,14880372,786.1134


In [8]:
names(gap_normal)
names(gapminder)

[1] "country"   "continent" "year"      "gdpPercap" "lifeExp"   "pop"

[1] "country"   "continent" "year"      "lifeExp"   "pop"       "gdpPercap"

Now we’ve got an intermediate dataframe gap_normal with the same dimensions as the original gapminder, but the order of the variables is different. Let’s fix that before checking if they are all.equal().

In [9]:
gap_normal <- gap_normal[,names(gapminder)]
all.equal(gap_normal,gapminder)

[1] TRUE

In [10]:
head(gap_normal)
head(gapminder)

country,continent,year,lifeExp,pop,gdpPercap
Afghanistan,Asia,1952,28.801,8425333,779.4453
Afghanistan,Asia,1957,30.332,9240934,820.8530
Afghanistan,Asia,1962,31.997,10267083,853.1007
Afghanistan,Asia,1967,34.020,11537966,836.1971
Afghanistan,Asia,1972,36.088,13079460,739.9811
Afghanistan,Asia,1977,38.438,14880372,786.1134


country,continent,year,lifeExp,pop,gdpPercap
Afghanistan,Asia,1952,28.801,8425333,779.4453
Afghanistan,Asia,1957,30.332,9240934,820.8530
Afghanistan,Asia,1962,31.997,10267083,853.1007
Afghanistan,Asia,1967,34.020,11537966,836.1971
Afghanistan,Asia,1972,36.088,13079460,739.9811
Afghanistan,Asia,1977,38.438,14880372,786.1134


We’re almost there, the original was sorted by country, continent, then year.

In [11]:
gap_normal <- gap_normal %>% arrange(country,continent,year)
all.equal(gap_normal,gapminder)

[1] TRUE

In [12]:
head(gap_normal)
head(gapminder)

country,continent,year,lifeExp,pop,gdpPercap
Afghanistan,Asia,1952,28.801,8425333,779.4453
Afghanistan,Asia,1957,30.332,9240934,820.8530
Afghanistan,Asia,1962,31.997,10267083,853.1007
Afghanistan,Asia,1967,34.020,11537966,836.1971
Afghanistan,Asia,1972,36.088,13079460,739.9811
Afghanistan,Asia,1977,38.438,14880372,786.1134


country,continent,year,lifeExp,pop,gdpPercap
Afghanistan,Asia,1952,28.801,8425333,779.4453
Afghanistan,Asia,1957,30.332,9240934,820.8530
Afghanistan,Asia,1962,31.997,10267083,853.1007
Afghanistan,Asia,1967,34.020,11537966,836.1971
Afghanistan,Asia,1972,36.088,13079460,739.9811
Afghanistan,Asia,1977,38.438,14880372,786.1134


That’s great! We’ve gone from the longest format back to the intermediate and we didn’t introduce any errors in our code.

## From long to wide format with tidyr::unite() and spread() 

Now lets convert the long data (gap_long) and make it wide. 

In [19]:
head(gap_long)
head(gap_wide)

country,continent,obs_type,year,obs_values
Afghanistan,Asia,gdpPercap,1952,779.4453
Albania,Europe,gdpPercap,1952,1601.0561
Algeria,Africa,gdpPercap,1952,2449.0082
Angola,Africa,gdpPercap,1952,3520.6103
Argentina,Americas,gdpPercap,1952,5911.3151
Australia,Oceania,gdpPercap,1952,10039.5956


country,continent,gdpPercap_1952,gdpPercap_1957,gdpPercap_1962,gdpPercap_1967,gdpPercap_1972,gdpPercap_1977,gdpPercap_1982,gdpPercap_1987,⋯,pop_1962,pop_1967,pop_1972,pop_1977,pop_1982,pop_1987,pop_1992,pop_1997,pop_2002,pop_2007
Afghanistan,Asia,779.4453,820.853,853.1007,836.1971,739.9811,786.1134,978.0114,852.3959,⋯,10267083,11537966,13079460,14880372,12881816,13867957,16317921,22227415,25268405,31889923
Albania,Europe,1601.0561,1942.284,2312.8890,2760.1969,3313.4222,3533.0039,3630.8807,3738.9327,⋯,1728137,1984060,2263554,2509048,2780097,3075321,3326498,3428038,3508512,3600523
Algeria,Africa,2449.0082,3013.976,2550.8169,3246.9918,4182.6638,4910.4168,5745.1602,5681.3585,⋯,11000948,12760499,14760787,17152804,20033753,23254956,26298373,29072015,31287142,33333216
Angola,Africa,3520.6103,3827.940,4269.2767,5522.7764,5473.2880,3008.6474,2756.9537,2430.2083,⋯,4826015,5247469,5894858,6162675,7016384,7874230,8735988,9875024,10866106,12420476
Argentina,Americas,5911.3151,6856.856,7133.1660,8052.9530,9443.0385,10079.0267,8997.8974,9139.6714,⋯,21283783,22934225,24779799,26983828,29341374,31620918,33958947,36203463,38331121,40301927
Australia,Oceania,10039.5956,10949.650,12217.2269,14526.1246,16788.6295,18334.1975,19477.0093,21888.8890,⋯,10794968,11872264,13177000,14074100,15184200,16257249,17481977,18565243,19546792,20434176


In [24]:
gap_temp <- gap_long %>% unite(col = "obs_type_year", obs_type, year, sep = "_")
head(gap_temp)

country,continent,obs_type_year,obs_values
Afghanistan,Asia,gdpPercap_1952,779.4453
Albania,Europe,gdpPercap_1952,1601.0561
Algeria,Africa,gdpPercap_1952,2449.0082
Angola,Africa,gdpPercap_1952,3520.6103
Argentina,Americas,gdpPercap_1952,5911.3151
Australia,Oceania,gdpPercap_1952,10039.5956


In [27]:
gap_long_to_wide <- gap_temp %>% spread(key = "obs_type_year", value = "obs_values")
head(gap_long_to_wide)

country,continent,gdpPercap_1952,gdpPercap_1957,gdpPercap_1962,gdpPercap_1967,gdpPercap_1972,gdpPercap_1977,gdpPercap_1982,gdpPercap_1987,⋯,pop_1962,pop_1967,pop_1972,pop_1977,pop_1982,pop_1987,pop_1992,pop_1997,pop_2002,pop_2007
Afghanistan,Asia,779.4453,820.853,853.1007,836.1971,739.9811,786.1134,978.0114,852.3959,⋯,10267083,11537966,13079460,14880372,12881816,13867957,16317921,22227415,25268405,31889923
Albania,Europe,1601.0561,1942.284,2312.8890,2760.1969,3313.4222,3533.0039,3630.8807,3738.9327,⋯,1728137,1984060,2263554,2509048,2780097,3075321,3326498,3428038,3508512,3600523
Algeria,Africa,2449.0082,3013.976,2550.8169,3246.9918,4182.6638,4910.4168,5745.1602,5681.3585,⋯,11000948,12760499,14760787,17152804,20033753,23254956,26298373,29072015,31287142,33333216
Angola,Africa,3520.6103,3827.940,4269.2767,5522.7764,5473.2880,3008.6474,2756.9537,2430.2083,⋯,4826015,5247469,5894858,6162675,7016384,7874230,8735988,9875024,10866106,12420476
Argentina,Americas,5911.3151,6856.856,7133.1660,8052.9530,9443.0385,10079.0267,8997.8974,9139.6714,⋯,21283783,22934225,24779799,26983828,29341374,31620918,33958947,36203463,38331121,40301927
Australia,Oceania,10039.5956,10949.650,12217.2269,14526.1246,16788.6295,18334.1975,19477.0093,21888.8890,⋯,10794968,11872264,13177000,14074100,15184200,16257249,17481977,18565243,19546792,20434176


### Question 3
**Take this 1 step further and create a gap_ludicrously_wide format data by spreading over countries, year and the 3 metrics? Hint this new dataframe should only have 5 rows.** Hint: your columns should look like this `gdpPercap_1952_Afghanistan`

In [31]:
gap_ludicrously_wide <- gap_long %>%
   unite(col = "var_names",obs_type,year,country,sep= "_") %>%
   spread(var_names,obs_values)

continent,gdpPercap_1952_Afghanistan,gdpPercap_1952_Albania,gdpPercap_1952_Algeria,gdpPercap_1952_Angola,gdpPercap_1952_Argentina,gdpPercap_1952_Australia,gdpPercap_1952_Austria,gdpPercap_1952_Bahrain,gdpPercap_1952_Bangladesh,⋯,pop_2007_Uganda,pop_2007_United Kingdom,pop_2007_United States,pop_2007_Uruguay,pop_2007_Venezuela,pop_2007_Vietnam,pop_2007_West Bank and Gaza,"pop_2007_Yemen, Rep.",pop_2007_Zambia,pop_2007_Zimbabwe
Africa,NA,NA,2449.008,3520.61,NA,NA,NA,NA,NA,⋯,29170398,NA,NA,NA,NA,NA,NA,NA,11746035,12311143
Americas,NA,NA,NA,NA,5911.315,NA,NA,NA,NA,⋯,NA,NA,301139947,3447496,26084662,NA,NA,NA,NA,NA
Asia,779.4453,NA,NA,NA,NA,NA,NA,9867.085,684.2442,⋯,NA,NA,NA,NA,NA,85262356,4018332,22211743,NA,NA
Europe,NA,1601.056,NA,NA,NA,NA,6137.076,NA,NA,⋯,NA,60776238,NA,NA,NA,NA,NA,NA,NA,NA
Oceania,NA,NA,NA,NA,NA,10039.6,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


## Key Points

The **tidyr** package is used to change the layout of dataframes.  
**gather()** to go from wide to long format.  
**spread()** to go from long to wide format.  

**seperate()** to split character strings into multiple variables.
**unite()** to